In [1]:
# import necessary librarries
from netCDF4 import Dataset
import netCDF4
import numpy as np
import math
import os

In [2]:
# Check that netcdf4 works
try: ncfile.close()
except: pass

# Create a new. Here we're creating a test netcf
ncfile = Dataset("test.nc",mode='w',format='NETCDF4_CLASSIC') # Don't put an entire folder path in the name section.
print(ncfile)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: 


In [3]:
# Adding dimensions to the netcdf file. Leave these be unless you want to increase the size of ensemble members, gridpoints, or the state vector 

member_dim = ncfile.createDimension('member', 80)
location_array = ncfile.createDimension('location', 40)
model_size = ncfile.createDimension('modelsize', 120)
time = ncfile.createDimension('time', None)

for dim in ncfile.dimensions.items():
    print(dim)

('member', <class 'netCDF4._netCDF4.Dimension'>: name = 'member', size = 80)
('location', <class 'netCDF4._netCDF4.Dimension'>: name = 'location', size = 40)
('modelsize', <class 'netCDF4._netCDF4.Dimension'>: name = 'modelsize', size = 120)
('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 0)


In [4]:
# Adding attributes to the netcdf file
ncfile.title = "an ensemble of spun-up model states"
ncfile.version = "$Id: $"
ncfile.model = "Lorenz_96_Tracer_Advection"
ncfile.model_forcing = 8.
ncfile.model_delta_t = 0.05
ncfile.source_rate = 100.
ncfile.exponential_sink_folding = 1.

In [6]:
# CREATING VARIABLES (THIS IS WHERE THE DATA GOES)

# Variable for Model Size
modelsize = ncfile.createVariable('modelsize', np.int32, ('modelsize',))
modelsize.dimension = 1.
modelsize.long_name = "the number of all the model variable"

In [8]:
# Variable for Location
location = ncfile.createVariable('location', np.double, ('location',))
location.short_name = "loc1d"
location.long_name = "location on a unit circle"
location.dimension = 1.
location.valid_range = 0.,1.

In [9]:
# Variable for State (Winds in our case)
state_var = ncfile.createVariable('state_variable', np.double, ('time','member','location',))
state_var.long_name = "The lorenz-96 wind variables"

# Variable for Tracer Concentration
tracer_conc = ncfile.createVariable('tracer_concentration', np.double, ('time','member','location',))
tracer_conc.long_name = "The lorenz-96 semi lagrangian tracers"

# Variable for Source Location and Strength
source = ncfile.createVariable('source', np.double, ('time','member','location',))
source.long_name = "The lorenz-96 semi lagrangian tracer sources"

In [10]:
# More variables for describe inflations (leave these be)
state_priorinf_mean = ncfile.createVariable('state_priorinf_mean', np.double, ('time','modelsize'))
state_priorinf_mean.long_name = "prior inflation value"

state_priorinf_sd = ncfile.createVariable('state_priorinf_sd', np.double, ('time','modelsize'))
state_priorinf_sd.long_name = "prior inflation standard deviation"

state_postinf_mean = ncfile.createVariable('state_postinf_mean', np.double, ('time','modelsize'))
state_postinf_mean.long_name = "posterior inflation value"

state_postinf_sd = ncfile.createVariable('state_postinf_sd', np.double, ('time','modelsize'))
state_postinf_sd.long_name = "posterior inflation standard deviation"

In [12]:
# Variable for Time

time = ncfile.createVariable('time', np.double, ('time',))
time.long_name = "valid time of the model state"
time.axis = "T"
time.cartesian_axis = "T"
time.calendar = "none"
time.unis = "days"

advance_to_time = ncfile.createVariable('advance_to_time', np.double)
advance_to_time.long_name = "desired time at the end of the next model advance"
advance_to_time.axis = "T"
advance_to_time.cartesian_axis = "T"
advance_to_time.calendar = "none"
advance_to_time.unis = "days"

In [13]:
print(ncfile)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    title: an ensemble of spun-up model states
    version: $Id: $
    model: Lorenz_96_Tracer_Advection
    model_forcing: 8.0
    model_delta_t: 0.05
    source_rate: 100.0
    exponential_sink_folding: 1.0
    dimensions(sizes): member(80), location(40), modelsize(120), time(0)
    variables(dimensions): int32 modelsize(modelsize), float64 location(location), float64 state_variable(time, member, location), float64 tracer_concentration(time, member, location), float64 source(time, member, location), float64 state_priorinf_mean(time, modelsize), float64 state_priorinf_sd(time, modelsize), float64 state_postinf_mean(time, modelsize), float64 state_postinf_sd(time, modelsize), float64 time(time), float64 advance_to_time()
    groups: 


In [15]:
x = np.genfromtxt('test.csv')
part_loc = np.arange(0,1,0.025)
one = np.ones(x.shape)
std_frac = 0.6*one

In [18]:
# Assigning values ot the variables

modelsize[:] = np.arange(1,121)
state_var[0,0,:] = x[0:40]
tracer_conc[0,0,:] = x[40:80]
source[0,0,:] = x[80:120]
v = np.arange(0,1,0.025)
location[:] = np.concatenate(part_loc, axis=None)
state_priorinf_mean[0,:] = one
state_priorinf_sd[0,:] = std_frac
state_postinf_mean[0,:] = one
state_postinf_sd[0,:] = std_frac
time[0] = 0
advance_to_time[0] = 0

In [19]:
ncfile.close()